In [ ]:
import cv2
import os


def read_video(output_folder, video_path):


    output_folder = video_path.split('/')[-1].split('.')[0]

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)


    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()


    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_interval = int(round(fps / 10))
    # print(frame_interval)

    current_frame = 0
    saved_frame = 0

    while True:
        ret, frame = cap.read()

        if ret:
            frame = frame[100:600, 600:1200]

            if current_frame % frame_interval == 0:

                filename = f"{output_folder}/frame_{saved_frame:04d}.jpg"
                cv2.imwrite(filename, frame)
                saved_frame += 1
            current_frame += 1
        else:
            break


    cap.release()
    print("Video processing completed. Frames saved to:", output_folder)


In [2]:
from tqdm import tqdm

file_path = '../us_raw/'
video_list = os.listdir(file_path)
print(video_list)
# for video in tqdm(video_list):
#     if video == 's03_cond3_2.mkv':
#         # continue
#         video_path = file_path + video
#         read_video(video_path)

['s01_cond3_1.mkv', 's01_cond3_2.mkv', 's02_cond3_1.mkv', 's02_cond3_2.mkv', 's03_cond3_1.mkv', 's03_cond3_2.mkv', 's04_cond3_1.mkv', 's04_cond3_2.mkv', 's05_cond3_1.mkv', 's05_cond3_2.mkv', 's06_cond3_1.mkv', 's06_cond3_2.mkv', 's07_cond3_1.mkv', 's07_cond3_2.mkv', 's08_cond3_1.mkv', 's08_cond3_2.mkv', 's11_cond3_1.mkv', 's11_cond3_2.mkv', 's12_cond3_1.mkv', 's12_cond3_2.mkv']


100%|██████████| 20/20 [00:06<00:00,  3.22it/s]

Video processing completed. Frames saved to: s03_cond3_2


In [51]:
import pandas as pd
import math
import numpy as np
import json

new_subject_list = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's11', 's12']
cond_list = ['1', '2']
json_dict = {}
x_radius = 6/13
y_radius = 5/6

for subject_name in new_subject_list:
    # print('dealing ' + subject_name)
    for cond in cond_list:
      # print('dealing ' + subject_name + ' ' + cond)
      file_path = f"us_annotation_March/{subject_name}_usraw_{cond}.csv"
      df = pd.read_csv(file_path)
      if subject_name == 's11' and cond == '1':
        df = df.loc[df['0.00E+00'] != 0]
      else:
        df = df.loc[df['0.000000000000000000e+00'] != 0]
      # df = df.loc[(df['0.000000000000000000e+00'] != 0) | (df['0.00E+00'] != 0)]
      df.columns = ['1x', '1y', '2x', '2y', '3x', '3y', '4x', '4y', '5x', '5y']
      df_new = pd.DataFrame(columns=['pitch_name', 'section', 'round', 'length_value_left', 'length_value_right', 'area_value_left', 'area_value_right', 'angle_value_in', 'angle_value_out', 'frame_num'])
      for index, row in df.iterrows():
          idx = index + 1
          # print(idx)
          # row['1x'], row['2x'], row['3x'], row['4x'], row['5x'] = row['1x'] / x_radius, row['2x'] / x_radius, row['3x'] / x_radius, row['4x'] / x_radius, row['5x'] / x_radius
          # row['1y'], row['2y'], row['3y'], row['4y'], row['5y'] = row['1y'] / y_radius, row['2y'] / y_radius, row['3y'] / y_radius, row['4y'] / y_radius, row['5y'] / y_radius
          row['1x'], row['2x'], row['3x'], row['4x'], row['5x'] = row['1x'], row['2x'], row['3x'], row['4x'], row['5x']
          row['1y'], row['2y'], row['3y'], row['4y'], row['5y'] = (row['1y']-100), (row['2y']-100), (row['3y']-100), (row['4y']-100), (row['5y']-100)
          json_dict[f'{subject_name}_{cond}_{idx}'] = [row['1x'], row['1y'], row['2x'], row['2y'], row['3x'], row['3y'], row['4x'], row['4y'], row['5x'], row['5y']]
          # print(row['1x'], row['2x'], row['3x'], row['4x'], row['5x'])
          # # baseline
          # if idx == anno.loc[(anno['name'] == subject_name) & (anno['cond'] == 'baseline'), 'start'].values[0]:
          #     # print(row['1x'])
          #     df_new.loc[len(df_new.index)] = ['', 'baseline', 0, length_value_left, length_value_right, area_value_left, area_value_right, angle_value_in, angle_value_out, idx]
          # # before
          # elif idx >= anno.loc[(anno['name'] == subject_name) & (anno['cond'] == 'after practice'), 'start'].values[0]:
          #     df_new.loc[len(df_new.index)] = ['', 'after practice', 4, length_value_left, length_value_right, area_value_left, area_value_right, angle_value_in, angle_value_out, idx]
print(len(json_dict))

with open('us_annotation_March.json', 'w') as f:
    json.dump(json_dict, f, indent=4)
  # print(len(df))
    # df_new.to_csv(subject_name + '_processed_post.csv', index=False)
    # print('saved ' + subject_name)

229.0 274.0 305.0 357.0 297.0
241.0 271.0 299.0 336.0 292.0
243.0 274.0 309.0 343.0 296.0
247.0 274.0 297.0 336.0 293.0
245.0 275.0 297.0 332.0 291.0
245.0 271.0 295.0 329.0 285.0
242.0 272.0 292.0 323.0 288.0
243.0 269.0 293.0 326.0 286.0
240.0 266.0 293.0 326.0 289.0
241.0 267.0 290.0 324.0 290.0
242.0 263.0 282.0 317.0 290.0
244.0 268.0 290.0 323.0 289.0
249.0 278.0 296.0 334.0 295.0
248.0 277.0 301.0 331.0 294.0
247.0 275.0 291.0 329.0 292.0
250.0 278.0 295.0 327.0 290.0
249.0 281.0 300.0 325.0 298.0
245.0 274.0 291.0 315.0 292.0
247.0 278.0 300.0 333.0 294.0
249.0 279.0 297.0 328.0 294.0
248.0 279.0 299.0 332.0 290.0
245.0 272.0 302.0 334.0 295.0
243.0 275.0 297.0 335.0 293.0
236.0 271.0 299.0 338.0 295.0
231.0 269.0 302.0 336.0 295.0
236.0 272.0 299.0 333.0 296.0
237.0 274.0 296.0 338.0 294.0
244.0 279.0 306.0 340.0 299.0
242.0 274.0 307.0 346.0 296.0
242.0 285.0 309.0 345.0 292.0
246.0 276.0 305.0 353.0 293.0
238.0 278.0 302.0 342.0 297.0
238.0 272.0 311.0 353.0 297.0
233.0 275.

In [48]:
# Normalize the data in the JSON file to the range [-1, 1]
import json 
import numpy as np
import torch

with open('us_annotation_March.json', 'r') as f:   
    json_dict = json.load(f) 
normalized_data = {}

# Function to normalize values to the range [-1, 1]
# def normalize(values):
#     min_val = min(values)
#     max_val = max(values)
#     print(min_val, max_val)
#     return [2 * (x - min_val) / (max_val - min_val) - 1 for x in values]

all_values = [value for sublist in json_dict.values() for value in sublist]
global_min = min(all_values)
global_max = max(all_values)
print(global_min, global_max)



# Apply normalization to each set of values in the JSON data
# for key, values in json_dict.items():
#     normalized_data[key] = normalize(values)
for key, values in json_dict.items():
    normalized_data[key] = [(x - global_min) / (global_max - global_min) * 2 - 1 for x in values]

# list(normalized_data.items())[:5]

with open('us_annotation_March_normal.json', 'w') as f:
    json.dump(normalized_data, f, indent=4)

86.0 520.0


In [49]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import json

with open('us_annotation_March_normal.json', 'r') as f:
    normalized_data = json.load(f)
    
class KeypointDataset(Dataset):
    def __init__(self, data):
        """
        Args:
            data (dict): Dictionary with normalized data.
        """
        self.keys = list(data.keys())
        self.data = data

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        # print(key)
        img = cv2.imread(f'./frames/{key.split("_")[0]}_cond3_{key.split("_")[1]}/frame_{int(key.split("_")[2]):04d}.jpg', cv2.IMREAD_GRAYSCALE)
        # print(img.shape)
        height_extension = 100
        new_height = img.shape[0] + height_extension
        new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
        new_image[:img.shape[0], :] = img
        new_image = cv2.resize(new_image, (512, 512))
        new_image = (new_image / 255.0) * 2 - 1
        # print(np.min(new_image), np.max(new_image))
        sample = self.data[key]

        return torch.tensor(sample, dtype=torch.float32), torch.tensor(new_image, dtype=torch.float32)


dataset = KeypointDataset(normalized_data)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)


for i, batch in enumerate(dataloader):
    # print(f"Batch {i+1}:")
    # print(batch)
    # 通常在这里添加模型训练或验证的代码
    break  # 这里我们只打印第一批数据，所以使用 break

In [90]:
import cv2 
import torch
import numpy as np

def draw_img(img, keypoint, gt_keypoint):
    # 将 keypoint 列表分解成 (x, y) 坐标对
    keypoint = ((keypoint + 1) / 2 ) * (global_max - global_min) + global_min
    gt_keypoint = ((gt_keypoint + 1) / 2 ) * (global_max - global_min) + global_min
    points = [(keypoint[i], keypoint[i+1]) for i in range(0, len(keypoint), 2)]
    gt_points = [(gt_keypoint[i], gt_keypoint[i+1]) for i in range(0, len(gt_keypoint), 2)]

    img = (img + 1) / 2 * 255
    img = cv2.resize(img, (600, 600))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    for x, y in gt_points:
        # 画圆圈标记关键点，指定圆心，半径，颜色和填充
        # x = x +100
        cv2.circle(img, (int(x), int(y)), radius=5, color=(150, 50, 0), thickness=-1)

    # 遍历所有点，使用cv2.circle在图像上画点
    for x, y in points:
        # 画圆圈标记关键点，指定圆心，半径，颜色和填充      
        cv2.circle(img, (int(x), int(y)), radius=5, color=(128, 255, 128), thickness=-1)
    

    # 返回标记了关键点的图像
    return img

# 从数据集中获取一个样本
sample = dataset[250]
keypoint = sample[0].numpy()
img = sample[1].numpy()
result = draw_img(img, keypoint, keypoint)
cv2.imwrite('result.png', result)

True

In [ ]:
import pandas as pd
import math
import numpy as np
import json

new_subject_list = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's11', 's12']
cond_list = ['1', '2']
json_dict = {}
x_radius = 6/13
y_radius = 5/6

for subject_name in new_subject_list:
    # print('dealing ' + subject_name)
    for cond in cond_list:
      # print('dealing ' + subject_name + ' ' + cond)
      file_path = f"us_annotation_March/{subject_name}_usraw_{cond}.csv"
      df = pd.read_csv(file_path)
      if subject_name == 's11' and cond == '1':
        df = df.loc[df['0.00E+00'] != 0]
      else:
        df = df.loc[df['0.000000000000000000e+00'] != 0]
      # df = df.loc[(df['0.000000000000000000e+00'] != 0) | (df['0.00E+00'] != 0)]
      df.columns = ['1x', '1y', '2x', '2y', '3x', '3y', '4x', '4y', '5x', '5y']
      df_new = pd.DataFrame(columns=['pitch_name', 'section', 'round', 'length_value_left', 'length_value_right', 'area_value_left', 'area_value_right', 'angle_value_in', 'angle_value_out', 'frame_num'])
      for index, row in df.iterrows():
          idx = index + 1
          # print(idx)
          # row['1x'], row['2x'], row['3x'], row['4x'], row['5x'] = row['1x'] / x_radius, row['2x'] / x_radius, row['3x'] / x_radius, row['4x'] / x_radius, row['5x'] / x_radius
          # row['1y'], row['2y'], row['3y'], row['4y'], row['5y'] = row['1y'] / y_radius, row['2y'] / y_radius, row['3y'] / y_radius, row['4y'] / y_radius, row['5y'] / y_radius
          row['1x'], row['2x'], row['3x'], row['4x'], row['5x'] = row['1x'], row['2x'], row['3x'], row['4x'], row['5x']
          row['1y'], row['2y'], row['3y'], row['4y'], row['5y'] = (row['1y']-100), (row['2y']-100), (row['3y']-100), (row['4y']-100), (row['5y']-100)
          json_dict[f'{subject_name}_{cond}_{idx}'] = [row['1x'], row['1y'], row['2x'], row['2y'], row['3x'], row['3y'], row['4x'], row['4y'], row['5x'], row['5y']]
          # print(row['1x'], row['2x'], row['3x'], row['4x'], row['5x'])
          # # baseline
          # if idx == anno.loc[(anno['name'] == subject_name) & (anno['cond'] == 'baseline'), 'start'].values[0]:
          #     # print(row['1x'])
          #     df_new.loc[len(df_new.index)] = ['', 'baseline', 0, length_value_left, length_value_right, area_value_left, area_value_right, angle_value_in, angle_value_out, idx]
          # # before
          # elif idx >= anno.loc[(anno['name'] == subject_name) & (anno['cond'] == 'after practice'), 'start'].values[0]:
          #     df_new.loc[len(df_new.index)] = ['', 'after practice', 4, length_value_left, length_value_right, area_value_left, area_value_right, angle_value_in, angle_value_out, idx]
print(len(json_dict))

with open('us_annotation_March.json', 'w') as f:
    json.dump(json_dict, f, indent=4)
  # print(len(df))
    # df_new.to_csv(subject_name + '_processed_post.csv', index=False)
    # print('saved ' + subject_name)